In [272]:
%%capture
#### Load Libraries (without cell output)

import autograd.numpy as np # import autograd wrapped numpy
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import sklearn
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import neighbors, metrics
from sklearn.preprocessing import scale

import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families import Gamma

In [273]:
claims = pd.read_csv("./Qualification_Package/Claims_Years_1_to_3.csv")
print(claims['claim_amount'].mean())

print(claims.shape)

1187.758451280604
(14173, 24)


In [1]:
claims.dtypes

NameError: name 'claims' is not defined

In [274]:
from pyampute.exploration.mcar_statistical_tests import MCARTest
mt = MCARTest(method="little")

In [275]:
object_columns = claims.select_dtypes(['float64', 'int64'])

In [277]:
object_columns.head(5)

,year,pol_no_claims_discount,pol_duration,drv_age1,drv_age_lic1,drv_age2,drv_age_lic2,vh_age,vh_speed,vh_value,vh_weight,population,town_surface_area,claim_amount
0,1,0.00,5,47,22,NaN,NaN,5,200.0,25000.0,1300.0,320,58.3,339.84
1,1,0.00,27,84,60,NaN,NaN,1,158.0,13390.0,885.0,530,385.6,1236.00
2,1,0.00,2,37,18,42.0,24.0,4,160.0,2088.0,0.0,960,701.5,224.33
3,1,0.00,26,48,27,49.0,27.0,1,182.0,11785.0,1080.0,1290,49.8,1343.41
4,1,0.07,2,26,8,NaN,NaN,3,250.0,25250.0,1450.0,70,30.8,130.23


In [276]:
print(mt(object_columns))

0.0


In [280]:
from scipy.stats import chi2

def little_mcar_test(data, alpha=0.05):
    """
    Performs Little's MCAR (Missing Completely At Random) test on a dataset with missing values.
    """
    data = pd.DataFrame(data)
    data.columns = ['x' + str(i) for i in range(data.shape[1])]
    data['missing'] = np.sum(data.isnull(), axis=1)
    n = data.shape[0]
    k = data.shape[1] - 1
    df = k * (k - 1) / 2
    chi2_crit = chi2.ppf(1 - alpha, df)
    chi2_val = ((n - 1 - (k - 1) / 2) ** 2) / (k - 1) / ((n - k) * np.mean(data['missing']))
    p_val = 1 - chi2.cdf(chi2_val, df)
    if chi2_val > chi2_crit:
        print(
            'Reject null hypothesis: Data is not MCAR (p-value={:.4f}, chi-square={:.4f})'.format(p_val, chi2_val)
        )
    else:
        print(
            'Do not reject null hypothesis: Data is MCAR (p-value={:.4f}, chi-square={:.4f})'.format(p_val, chi2_val)
        )

In [281]:
print(little_mcar_test(object_columns))

Reject null hypothesis: Data is not MCAR (p-value=0.0000, chi-square=870.5631)
None


CONCLUSION: Since data is not missing at random, we use KNN to predict missing values before implementing regressor.